In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.12,-134.97,78.67,66,24,7.99,few clouds
1,1,Puerto Ayora,EC,-0.74,-90.35,82.40,61,0,8.05,clear sky
2,2,Ketchikan,US,55.34,-131.65,42.80,93,90,19.46,moderate rain
3,3,Ushuaia,AR,-54.80,-68.30,55.40,69,40,21.92,light rain
4,4,Jumla,NP,29.27,82.18,37.27,55,8,3.69,clear sky


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.12,-134.97,78.67,66,24,7.99,few clouds
1,1,Puerto Ayora,EC,-0.74,-90.35,82.40,61,0,8.05,clear sky
10,10,Toma,BF,12.77,-2.88,79.32,18,0,6.64,clear sky
12,12,Porto Velho,BR,-8.76,-63.90,77.00,94,40,2.24,thunderstorm
13,13,Hilo,US,19.73,-155.09,80.60,61,20,6.93,light rain
18,18,Faanui,PF,-16.48,-151.75,80.42,76,6,17.00,clear sky
19,19,Yulara,AU,-25.24,130.99,78.80,39,64,19.46,broken clouds
21,21,Banjar,ID,-8.19,114.97,78.80,94,75,3.36,broken clouds
24,24,Coahuayana,MX,18.73,-103.68,80.60,32,5,9.17,clear sky
25,25,Saint-Philippe,RE,-21.36,55.77,77.00,69,0,17.22,light rain


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                208
City                   208
Country                207
Lat                    208
Lng                    208
Max Temp               208
Humidity               208
Cloudiness             208
Wind Speed             208
Current Description    208
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
new_preferred_cities_df = preferred_cities_df.dropna()
new_preferred_cities_df.count()

City_ID                207
City                   207
Country                207
Lat                    207
Lng                    207
Max Temp               207
Humidity               207
Cloudiness             207
Wind Speed             207
Current Description    207
dtype: int64

In [70]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = new_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,78.67,few clouds,-23.12,-134.97,
1,Puerto Ayora,EC,82.40,clear sky,-0.74,-90.35,
10,Toma,BF,79.32,clear sky,12.77,-2.88,
12,Porto Velho,BR,77.00,thunderstorm,-8.76,-63.90,
13,Hilo,US,80.60,light rain,19.73,-155.09,
...,...,...,...,...,...,...,...
674,Manokwari,ID,81.07,moderate rain,-0.87,134.08,
677,Robertsport,LR,80.73,clear sky,6.75,-11.37,
679,Bom Jesus,BR,78.24,moderate rain,-9.07,-44.36,
683,Altamira,BR,75.81,overcast clouds,-3.20,-52.21,


In [98]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
idx = 0
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.iloc[idx, -1] = hotels["results"][0]["name"]
        

    except:
        print("Hotel not found... skipping.")
    idx = idx + 1

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [99]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,78.67,few clouds,-23.12,-134.97,Pension Maro'i
1,Puerto Ayora,EC,82.40,clear sky,-0.74,-90.35,Finch Bay Galapagos Hotel
10,Toma,BF,79.32,clear sky,12.77,-2.88,Auberge de l'amitié
12,Porto Velho,BR,77.00,thunderstorm,-8.76,-63.90,Hotel Vila Rica Porto Velho
13,Hilo,US,80.60,light rain,19.73,-155.09,Hilo Hawaiian Hotel
...,...,...,...,...,...,...,...
674,Manokwari,ID,81.07,moderate rain,-0.87,134.08,Swiss Belhotel Manokwari
677,Robertsport,LR,80.73,clear sky,6.75,-11.37,Kwepunha Retreat
679,Bom Jesus,BR,78.24,moderate rain,-9.07,-44.36,Hotel Brasão & Floricultura Bella Flor
683,Altamira,BR,75.81,overcast clouds,-3.20,-52.21,Requinte Hotel


In [100]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"]!=""]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,78.67,few clouds,-23.12,-134.97,Pension Maro'i
1,Puerto Ayora,EC,82.40,clear sky,-0.74,-90.35,Finch Bay Galapagos Hotel
10,Toma,BF,79.32,clear sky,12.77,-2.88,Auberge de l'amitié
12,Porto Velho,BR,77.00,thunderstorm,-8.76,-63.90,Hotel Vila Rica Porto Velho
13,Hilo,US,80.60,light rain,19.73,-155.09,Hilo Hawaiian Hotel
18,Faanui,PF,80.42,clear sky,-16.48,-151.75,Bora-Bora Pearl Beach Resort & Spa
19,Yulara,AU,78.80,broken clouds,-25.24,130.99,Desert Gardens Hotel - Ayers Rock Resort
21,Banjar,ID,78.80,broken clouds,-8.19,114.97,The Hamsa Resort
24,Coahuayana,MX,80.60,clear sky,18.73,-103.68,Hotel Los Arcos
25,Saint-Philippe,RE,77.00,light rain,-21.36,55.77,"Chambres d'hôte ""La Trinité"""


In [101]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [102]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [103]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))